![img](https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png) Filippo Miatto (2024) 

# Lecture 1: intro and tools



---

#### After going through this lecture you will be able to:
1. Describe the mathematical representation of quantum states
2. Use some advanced tensor tools from `numpy`
3. Visualize qubits on the Bloch sphere
4. Reason about wave functions

---
***Summary of Lecture 1:***
_Today we introduce quantum **States**: the first of the three pillars of Quantum Mechanics (States, Transformations and Measurements). Quantum states are unit vectors in a complex vector space (Hilbert space) and they describe how a quantum system "is". As quantum states are vectors, they can be linearly combined to form other valid states, which are referred to as "superpositions".
Qubits are the simplest possible quantum systems because their Hilbert space is only 2-dimensional, and therefore their states can be visualized on the Bloch sphere. More complex systems have states that are defined on higher-dimensional spaces, even infinite-dimensional spaces (in which case it makes more sense to talk about wavefunctions)._

---

# 0. Introduction
Quantum Mechanics is the theory that describes the most fundamental building blocks of everything. The typical systems that it applies to are atoms, particles, fields etc... and in principle it could apply to much larger systems. The difficulty with applying quantum mechanics to large systems is that the number of degrees of freedom involved grows linearly, but the complexity of the quantum mechanical description grows exponentially in the number of degrees of freedom. For this reason QM practically focuses on fundamental systems, but there is no theoretical limit to the size of the systems that it applies to.

The only limit is our ability to use quantum mechanics, not its correctness.

# 1. It's just linear algebra

Quantum Mechanics is based on linear algebra. Whenever you feel you're struggling, you should remind yourself that "it's just linear algebra!".

In QM we have three main classes of objects: States, Transformations and Measurements. All of them can be described by vectors, matrices, and/or higher order tensors. So no matter how complicated things may seem, it's all just linear algebra.

In this first section we will set up a few important concepts and we will get you started with some useful functions and numpy tools.

## 1.1 Complex vector spaces
One important aspect of QM is that the vector spaces that we use are complex, i.e. the entries of the vectors are complex numbers. To help with clarity, we can indicate our complex vector space as $\mathbb{C}^n$ where we have fixed its dimension to $n$. We will encounter vector spaces of various dimension such as $\mathbb{C}^2$ (the Hilbert space of a qubit). Let's now see a couple of fundamental operations that we will often need in QM: inner products and norms.

As our vector spaces are Hilbert spaces, they already come equipped with an inner product:

$$
\langle \mathbf{v}, \mathbf{w}\rangle = \sum_iv_i^*w_i
$$

notice that the vector $\bf v$ is conjugated when computing the inner product, i.e. we flip the sign of the imaginary component. 

Then the inner product can be used to define a norm (that is why all inner product spaces are also normed spaces):

$$
||\mathbf{v}|| = \sqrt{\langle \mathbf{v}, \mathbf{v}\rangle} = \sqrt{\sum_iv_i^*v_i}
$$

## 1.2 Dirac's notation
There is a particular notation that has been adopted by the scientific community, known as Dirac's bra-ket notation. It comes from the alternative way of writing an inner product between two vectors $\mathbf{v}$ and $\mathbf{w}$ as $\langle \mathbf{v}|\mathbf{w}\rangle$. If you interpret it as two objects coming together and "sticking" to each other, you can separate them into $\langle \mathbf{v}|$ and $|\mathbf{w}\rangle$ and use them as row and column vectors (more appropriately as covectors and vectors). We call them "bra" and "ket" because together they sound like the word "bracket". Whatever we write inside the ket (or the bra) is just a label for that (co)vector, which sometimes carries useful information about the quantum state that it labels, like when we express the state of electromagnetic vacuum as $|0\rangle$, meaning zero photons.

As the vector space is complex, remember that to turn vectors and covectors into each other you also take the complex conjugate of the elements: $\langle\psi| = |\psi\rangle^\dagger$ where the dagger symbol $\dagger$ computes the "hermitian conjugate" (i.e. the conjugate transpose).

We will make abundant use of inner products and norms. So let's begin our first activity by defining a couple of helper functions to compute inner products and norms.

### Activity 1: inner products and norms
- Write a function $f(\mathbf{v}, \mathbf{w})$ that computes the inner product between two vectors $\mathbf{v}$ and $\mathbf{w}$. The signature should be `f(array[complex], array[complex]) -> complex`
- Write a function $f(\mathbf{v})$ that computes the norm of a vector $\mathbf{v}$. The signature should be `f(array[complex]) -> float`

Note that we are not necessarily seeking optimal performance here, exactness is enough.

In [ ]:
import numpy as np

def inner_prod(v, w):
    return np.sum(np.conj(v) * w)

def norm(v):
    return np.sqrt(inner_prod(v,v))

: 

## 1.3 A bit of useful numpy

`Numpy` is one of the most popular python libraries for numerical calculus.

The most useful tool that I want to teach you is the function `np.einsum()`, but before we get there let's learn about tensors and axes. Tensors are generalizations of vectors and matrices. For us a tensor is a collection of numbers, where each number is identified by a set of integer coordinates. It's also called an "array" in `numpy`.

The meaning that we attribute to the array depends on what we will use it for. Here's a couple examples:

1. A complex tensor with a single index of dimension $n$ can be interpreted as a vector $T \in \mathbb{C}^n$.

2. A complex tensor with two indices of dimension $m$ and $n$ can be interpreted as a matrix, i.e. a map $T : \mathbb{C}^n\rightarrow \mathbb{C}^m$, but also as a vector $T \in \mathbb{C}^m\otimes \mathbb{C}^n$.

A general tensor is simply an object with multiple indices: $T_{ijklmn\dots}$, whose meaning depends on the context.

The number of indices is called the _rank_ of the tensor, so column vectors are rank-1 tensors, matrices are rank-2 tensors etc...

Each index has a dimension (i.e. the number of distinct integer values that it can have), and the dimension does not have to be the same for all the indices of a tensor, e.g. a rectangular matrix has two indices of different dimensions. If we call $d(j)$ the dimension of the index $j$, then a tensor $T_{j_1,\dots,j_r}$ of rank $r$ contains $d(j_1)\times d(j_2)\times\dots\times d(j_r)$ numbers, and so the size of a tensor scales more or less exponentially with the number of indices, i.e. with the rank.

`numpy` has many useful functions to deal with vectors and matrices, but it can also easily deal with higher-order tensors. Here are a few useful functions and methods to know:

- `np.newaxis`: this method allows us to create new indices (of dimension 1) for a tensor. Notice that adding a new index of dimension 1 does not change how many numbers are in the tensor (if it were of dimension > 1, then the size of the tensor would have to change).
- `Ellipsis` (three dots): this object is a placeholder for any number of indices. Very useful when we want to index a tensor with a variable number of indices.
- Broadcasting: this automatic operation allows us to define computations between tensors of seemingly "incompatible" shape and save a few keystrokes.

In [ ]:
# Usage of np.newaxis

v = np.array([1,2,3]) # a vector

v1 = v[:, np.newaxis]
print(f'Adding a second index. New shape of the tensor is {v1.shape}:\n', v1, '\n')

v2 = v[np.newaxis, :]
print(f'Adding a new first index. New shape of the tensor is {v2.shape}:\n', v2, '\n')

v3 = v[:, np.newaxis, np.newaxis]
print(f'Adding a second and third index. New shape of the tensor is {v3.shape}:\n', v3, '\n')

In [ ]:
# using new axes for broadcasting

v = np.array([1,2,3]) # a 3-dim vector
m = np.ones((3,3)) # a 3x3 matrix of ones

# compare:

print('broadcasting along 2nd index (rows):')
print(v[np.newaxis, :] * m, '\n')

print('broadcasting along 1st index (columns):')
print(v[:, np.newaxis] * m)

In [ ]:
# Usage of ellipsis

T = np.zeros((2,3,4,5)) # tensor of shape (2,3,4,5)
T[..., np.newaxis].shape  # ... represents (2,3,4,5)

## 1.4 `np.einsum()`
Now we can talk about `np.einsum()`, which is an extremely useful function. It can handle all sorts of tensor products, transposition, traces and much more.

It takes as first argument a string that explains what happens to the indices (indicated as letters) and then it takes as many tensors as required by the string. There is only one rule:

$$
\mathbf{Repeated\ indices\ are\ summed\ over}
$$

This means that if we use the same symbol (the same letter) for an index, we mean that in the expression there should be a summation over that index. Let's see a few examples.

### Matrix multiplication
Matrix multiplication is:

$$
(MN)_{ik} = \sum_j M_{ij}N_{jk} =: M_{ij}N_{jk}
$$

In the last step we use what is called "Einstein's summation convention", where we omit writing the summation symbol $\sum_j$, because $j$ is repeated (it appears in both $M$ and $N$) and therefore it's being summed over. When we sum over a repeated index we say that we "_contract" that index_.

With `np.einsum` matrix multiplication would be:

```python
np.einsum('ij,jk -> ik', M, N)
```

Matrix multiplication between, say, 4 matrices is
$$
(MNPQ)_{im} = \sum_{jkl} M_{ij}N_{jk}P_{kl}Q_{lm} \equiv M_{ij}N_{jk}P_{kl}Q_{lm}
$$
With `np.einsum` this would be:

```python
np.einsum('ij,jk,kl,lm -> im', M, N, P, Q)
```

### Higher order contractions
This can obviously work for tensors of any order! Here's a random example that does not mean anything:

$$
T_{m} = \sum_{jkl} M_{j}N_{jklm}P_{jk}Q_{l} \equiv M_{j}N_{jklm}P_{jk}Q_{l}
$$

(note that $m$ is the only index here which is never repeated, so the final result must be a vector indexed by $m$)

With `np.einsum` this would be:

```python
np.einsum('j,jklm,jk,l -> m', M, N, P, Q)
```

### Transposition
The string after the arrow allows us to do some final rearranging of the indices, like a transposition:

$$
(MN)^T_{ki} = (MN)_{ik}
$$
With `np.einsum` this would be:

```python
np.einsum('ij,jk -> ki', M, N) # notice: ki and not ik
```

### Traces
If we repeat an index belonging to the same tensor, we compute a trace:

$$
Tr(M) = \sum_{i} M_{ii} \equiv M_{ii}
$$
With `np.einsum` this would be:

```python
np.einsum('ii', M)
```

### Tensor products (i.e. outer products)
Outer products are the opposite of inner products, i.e. we simply juxtapose indices:

E.g. with vectors:
$$
\mathbf{v}\otimes \mathbf{w} = v_{i}w_{j}
$$
With `np.einsum` this would be:

```python
np.einsum('i,j -> ij', v, w)
```

Or with matrices:
$$
M\otimes N = M_{ij}N_{kl}
$$
With `np.einsum` this would be:

```python
np.einsum('ij,kl -> ijkl', M, N)
```

Usually it is a bad idea to "actually" calculate an outer product. For better performance keep the parts separated unless the values of the outer product are needed. Chances are that they might get simpler by contracting with other tensors, as the calculation progresses.

### Activity 2: A better Hilbert-Schmidt inner product

The Hilbert-Schmidt inner product is an inner product between complex matrices and it is defined as follows:

$$
\langle M, N\rangle = Tr(M^\dagger N)
$$

where $M^\dagger$ means we transpose and complex-conjugate $M$.

- `v1`: implement the formula as is written above, using `np.matmul`, `np.trace` and `np.conj` to compute the conjugate
- `v2`: implement the formula using `np.einsum` and `np.conj`
- `v3` (Bonus): implement the formula in a more efficient way than `v1` without using `np.einsum` (tip: it looks the same as the function `inner_prod` that you wrote in Activity 1)

In [10]:
# v1
def HS_inner_product(M, N):
    return np.trace(np.matmul(np.transpose(np.conj(M)), N))

# v2
def HS_inner_product(M, N):
    return np.einsum('ji,ji', np.conj(M), N)

# v3
def HS_inner_product(M, N):
    return np.sum(np.conj(M) * N)

Note how numpy's syntax allows us to write the inner product in a way that can apply to tensors of any (matching) shape but still look like the formula for the inner product between two vectors.


### Activity 3: A better way to multiply by a diagonal matrix

Consider the product between three matrices: $ABC$, where $B$ is a diagonal matrix. This happens all the time when we consider eigenvalue decomposition or a singular value decomposition, where the central matrix is diagonal.

- `v1`: implement this product as written above (treat $B$ as if it were any matrix) using `np.einsum`
- `v2`: implement this product in a more efficient way (use the fact that $B$ is diagonal) using `np.einsum` 
- `v3`: implement this product with `np.einsum` and `np.diag(B)`.

In [ ]:
# v1
def prod(A, B, C):
    return np.einsum('ij,jk,kl -> il', A, B, C) # 3 matrices
# v2
def prod(A, B, C):
    return np.einsum('ij,jj,jl -> il', A, B, C) # B is diagonal
# v3
def prod(A, B, C):
    return np.einsum('ij,j,jl -> il', A, np.diag(B), C) # only using the diagonal of B